In [1]:
!python3 -m pip install pandas
!python3 -m pip install sklearn
!pip install ray[tune]

     |████████████████████████████████| 26.7 MB 20.3 MB/s eta 0:00:01    |█                               | 839 kB 10.6 MB/s eta 0:00:03     |██▏                             | 1.8 MB 10.6 MB/s eta 0:00:03     |██████▊                         | 5.6 MB 10.6 MB/s eta 0:00:02     |████████                        | 6.8 MB 10.6 MB/s eta 0:00:02     |████████████████▉               | 14.0 MB 10.6 MB/s eta 0:00:02
     |████████████████████████████████| 41.6 MB 24 kB/s s eta 0:00:01    |██                              | 2.7 MB 20.8 MB/s eta 0:00:02     |███▍                            | 4.3 MB 20.8 MB/s eta 0:00:02     |████▏                           | 5.4 MB 20.8 MB/s eta 0:00:02     |█████                           | 6.6 MB 20.8 MB/s eta 0:00:02     |█████▉                          | 7.6 MB 20.8 MB/s eta 0:00:02     |███████▍                        | 9.7 MB 20.8 MB/s eta 0:00:02     |██████████████▏                 | 18.4 MB 20.8 MB/s eta 0:00:02     |███████████████▏                | 19.7 

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sklearn

# Your task now is to use scikit-learn to train a RandomForestClassifier https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html  for predicting the forest cover type. The dataset can be downloaded directly via the sklearn API: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_covtype.html#sklearn.datasets.fetch_covtype and there are several notebooks on Kaggle where you can learn about the data and background to the dataset: https://www.kaggle.com/c/forest-cover-type-prediction.

# Do the following:
# 1.	Set up the problem using sklearn. Explore the parameters used in the Random Forest implementation. Hint: use the get_param() method. Train the model using the default parameters.

# 2.	Implement a distributed hyperparameter tuning pipeline for tuning the parameters ‘max_depth’, ‘n_estimators’ and ‘ccp_alpha’ using the RayTune framework and an exhaustive Grid Search strategy. Configure, deploy and run it on up to 3 VMs of “small” flavor.

In [3]:
trainDF = pd.read_csv('train.csv')
#testDF = pd.read_csv('test.csv') - this file doesn't actually contain the target variable of soil type.

trainDF.head()

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [4]:
trainFeatures = trainDF.iloc[:, :-1] # all rows, all but last column
trainClasses = trainDF.iloc[:, -1] # all rows, only last column

In [5]:
RFclassifier = RandomForestClassifier()
params = RFclassifier.get_params()

In [6]:
#fitted_model = RFclassifier.fit(trainFeatures, trainClasses)

sklearn.model_selection.cross_validate(RFclassifier, trainFeatures, trainClasses)

{'fit_time': array([2.45511031, 2.26639366, 2.30799031, 2.35039783, 2.32255745]),
 'score_time': array([0.06980276, 0.09295368, 0.1009872 , 0.08324575, 0.0939579 ]),
 'test_score': array([0.45734127, 0.5952381 , 0.67724868, 0.73181217, 0.77579365])}

In [3]:
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [4]:
ray.init(address='auto')



2022-04-30 13:39:38,403	INFO worker.py:946 -- Connecting to existing Ray cluster at address: 192.168.2.133:6379


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.10', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '192.168.2.133', 'raylet_ip_address': '192.168.2.133', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-04-30_12-16-26_345962_1239/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-04-30_12-16-26_345962_1239/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-04-30_12-16-26_345962_1239', 'metrics_export_port': 47081, 'gcs_address': '192.168.2.133:6379', 'address': '192.168.2.133:6379', 'node_id': '0e93e228dd070a46a831f215df5e99ee9128a9de874f2d047590f32e'})

In [5]:
search_space = {
    "max_depth": tune.grid_search([5*i for i in range(1,10)]),
    "n_estimators": tune.grid_search([10,100,200,300]),
    "ccp_alpha": tune.grid_search([0.0,0.1,0.2]),
}

In [6]:
import statistics

def train(config):
    RFclassifier = RandomForestClassifier(**config)
    params = RFclassifier.get_params()
    xval = sklearn.model_selection.cross_validate(RFclassifier, trainFeatures, trainClasses)
    tune.report(mean_accuracy=statistics.mean(xval["test_score"]))
    

In [ ]:
analysis = tune.run(train, config=search_space)

2022-04-30 13:40:03,161	WARNING function_runner.py:598 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-04-30 13:40:08,023	INFO trial_runner.py:803 -- starting train_4651e_00000


Trial name,status,loc,ccp_alpha,max_depth,n_estimators
train_4651e_00000,RUNNING,192.168.2.133:1964,0,5,10
train_4651e_00001,PENDING,,0.1,5,10
train_4651e_00002,PENDING,,0.2,5,10
train_4651e_00003,PENDING,,0,10,10
train_4651e_00004,PENDING,,0.1,10,10
train_4651e_00005,PENDING,,0.2,10,10
train_4651e_00006,PENDING,,0,15,10
train_4651e_00007,PENDING,,0.1,15,10
train_4651e_00008,PENDING,,0.2,15,10
train_4651e_00009,PENDING,,0,20,10


(train pid=1964) 2022-04-30 13:40:10,472	ERROR function_runner.py:281 -- Runner Thread raised error.
(train pid=1964) Traceback (most recent call last):
(train pid=1964)   File "/usr/local/lib/python3.8/dist-packages/ray/tune/function_runner.py", line 272, in run
(train pid=1964)     self._entrypoint()
(train pid=1964)   File "/usr/local/lib/python3.8/dist-packages/ray/tune/function_runner.py", line 348, in entrypoint
(train pid=1964)     return self._trainable_func(
(train pid=1964)   File "/usr/local/lib/python3.8/dist-packages/ray/util/tracing/tracing_helper.py", line 462, in _resume_span
(train pid=1964)     return method(self, *_args, **_kwargs)
(train pid=1964)   File "/usr/local/lib/python3.8/dist-packages/ray/tune/function_runner.py", line 640, in _trainable_func
(train pid=1964)     output = fn()
(train pid=1964)   File "/tmp/ipykernel_1744/424335076.py", line 4, in train
(train pid=1964) NameError: name 'RandomForestClassifier' is not defined
(train pid=1964) Exception in thr

Result for train_4651e_00000:
  date: 2022-04-30_13-40-10
  experiment_id: 3208b94f17804cccb58df8b681e74324
  hostname: patrick-c3-1
  node_ip: 192.168.2.133
  pid: 1964
  timestamp: 1651318810
  trial_id: 4651e_00000
  


Trial name,status,loc,ccp_alpha,max_depth,n_estimators
train_4651e_00001,PENDING,,0.1,5,10
train_4651e_00002,PENDING,,0.2,5,10
train_4651e_00003,PENDING,,0,10,10
train_4651e_00004,PENDING,,0.1,10,10
train_4651e_00005,PENDING,,0.2,10,10
train_4651e_00006,PENDING,,0,15,10
train_4651e_00007,PENDING,,0.1,15,10
train_4651e_00008,PENDING,,0.2,15,10
train_4651e_00009,PENDING,,0,20,10
train_4651e_00010,PENDING,,0.1,20,10


...

2022-04-30 13:55:16,850	WARNING util.py:171 -- The `on_step_begin` operation took 1.349 s, which may be a performance bottleneck.
2022-04-30 13:55:23,584	WARNING util.py:171 -- The `on_step_end` operation took 0.770 s, which may be a performance bottleneck.


Trial name,status,loc,ccp_alpha,max_depth,n_estimators
train_4651e_00007,PENDING,,0.1,15,10
train_4651e_00008,PENDING,,0.2,15,10
train_4651e_00009,PENDING,,0,20,10
train_4651e_00010,PENDING,,0.1,20,10
train_4651e_00011,PENDING,,0.2,20,10
train_4651e_00012,PENDING,,0,25,10
train_4651e_00013,PENDING,,0.1,25,10
train_4651e_00014,PENDING,,0.2,25,10
train_4651e_00015,PENDING,,0,30,10
train_4651e_00016,PENDING,,0.1,30,10


Trial name,status,loc,ccp_alpha,max_depth,n_estimators
train_4651e_00007,PENDING,,0.1,15,10
train_4651e_00008,PENDING,,0.2,15,10
train_4651e_00009,PENDING,,0,20,10
train_4651e_00010,PENDING,,0.1,20,10
train_4651e_00011,PENDING,,0.2,20,10
train_4651e_00012,PENDING,,0,25,10
train_4651e_00013,PENDING,,0.1,25,10
train_4651e_00014,PENDING,,0.2,25,10
train_4651e_00015,PENDING,,0,30,10
train_4651e_00016,PENDING,,0.1,30,10


2022-04-30 13:55:35,841	WARNING util.py:171 -- The `on_step_begin` operation took 4.600 s, which may be a performance bottleneck.


Trial name,status,loc,ccp_alpha,max_depth,n_estimators
train_4651e_00007,PENDING,,0.1,15,10
train_4651e_00008,PENDING,,0.2,15,10
train_4651e_00009,PENDING,,0,20,10
train_4651e_00010,PENDING,,0.1,20,10
train_4651e_00011,PENDING,,0.2,20,10
train_4651e_00012,PENDING,,0,25,10
train_4651e_00013,PENDING,,0.1,25,10
train_4651e_00014,PENDING,,0.2,25,10
train_4651e_00015,PENDING,,0,30,10
train_4651e_00016,PENDING,,0.1,30,10
